# Flood Adaptation: Minimal Model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as a simplified representation of household adaptation to flood events in a social network context. The model serves as a template for understanding the fundamental mechanics of flood adapdtation with a given social network.

This directory is structured as follows:
 - A Mesa Agent class `Households` is defined in `agents.py`
 - A Mesa Agent class `Government` is defined in `agents.py` (this agent currently does not perform any action)
 - A Mesa Model class `AdaptationModel` is defined in `model.py`
 - This notebook `demo.ipynb` provides a demonstration of the model and its visualization.

Each household agent in the model represents a household with attributes and behaviors that determine its response to flood events. The flood depth attribute assigned to each household is currently random for demonstration purposes. The decision by each household to adapt is also arbitrary. In a more comprehensive model, this could be replaced with more sophisticated decision-making algorithms.

The current implementation includes several simplifications and arbitrary choices, such as random flood depths and basic adaptation logic based on random thresholds. These aspects are designed for clarity and ease of understanding and should be replaced with realistic and sound choices.

You are encouraged to modify, expand, and customize the model. This might include integrating real-world data, implementing more complex decision-making processes for agents, or exploring different network topologies and their effects on social dynamics.

The visualisation provided is basic. Additional aspects can be added for visualization and more in-depth analysis.

_This notebook was tested on 2023-11-13 with Mesa version 2.1.4 on Python 3.12.0._

# Running the model and visualizing the model domain and social network over time

In [12]:
import random

from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx

# Determine the input values for your network and dynamic parameters and how they should be distributed throughout the population.
network_dynamics_dictionary = {
                # The social network structure that is used. Can currently be
                # "erdos_renyi", "barabasi_albert", "watts_strogatz", or "no_network"
                "network": 'watts_strogatz',
                "number_of_households": 50,  # number of household agents
                "probability_of_network_connection": 0.4,  # likeliness of edge being created between two nodes
                "number_of_edges": 3,  # number of edges for BA network
                "number_of_nearest_neighbours": 5,  # number of nearest neighbours for WS social network
                "household_tolerance": 0.1,  # the tolerance level for each agent,
                "flood_time_tick": 5,  # time of flood
                "seed": 1  # The seed used to generate pseudo random numbers
            }

# Determine the input values for your agent's attributes and how they should be distributed throughout the population.
attribute_dictionary = {
                "wealth": [0.2, 'UI', (0, 3)],
                "has_child": [0.2, 'B', (0.2)],
                "house_size": [0.05, 'UI', (0, 2)],
                "house_type": [0.05, 'UI', (0, 1)],
                "education_level": [0.05, 'UI', (0, 3)],
                "social_preference": [0.2, 'U', (-1, 1)],
                "age": [0.2, 'N', (33.4, 5)]
            }

# Defining the standard values for agent interaction dynamics in case none are provided.
agent_interaction_dictionary = {
                "household_tolerance": 0.3,  # the tolerance level for each agent
                "bias_change_per_tick": 0.2,  # Bias change per tick when an agent when it's influenced by its network
                "probability_positive_bias_change": 1,  # Probability that agent changes it's bias positively
                "probability_negative_bias_change": 0.05,  # Probability that agent changes it's bias negatively
                "adaption_threshold": 0.9  # Threshold of bias an agent needs to adapt
            }

# Initialize the Adaptation Model with 50 household agents.
model = AdaptationModel(network_dynamics_dictionary=network_dynamics_dictionary, attribute_dictionary=attribute_dictionary, agent_interaction_dictionary= agent_interaction_dictionary)
# flood_map_choice can be "harvey", "100yr", or "500yr"

# Calculate positions of nodes for the network plot.
# The spring_layout function positions nodes using a force-directed algorithm,
# which helps visualize the structure of the social network.
pos = nx.spring_layout(model.G)

# Define a function to plot agents on the network.
# This function takes a matplotlib axes object and the model as inputs.
# def plot_network(ax, model):
#     # Clear the current axes.
#     ax.clear()
#     # Determine the color of each node (agent) based on their adaptation status.
#     colors = ['blue' if agent.is_adapted else 'red' for agent in model.schedule.agents]
#     # Draw the network with node colors and labels.
#     nx.draw(model.G, pos, node_color=colors, with_labels=True, ax=ax)
#     # Set the title of the plot with the current step number.
#     ax.set_title(f"Social Network State at Step {model.schedule.steps}", fontsize=12)

# Generate the initial plots at step 0.
# Plot the spatial distribution of agents. This is a function written in the model.py
# model.plot_model_domain_with_agents()

# Plot the initial state of the social network.
# fig, ax = plt.subplots(figsize=(7, 7))
# plot_network(ax, model)
# plt.show()

# Run the model for 20 steps and generate plots every 5 steps.
for step in range(20):
    model.step()
# 
#     # Every 5 steps, generate and display plots for both the spatial distribution and network.
#     # Note the first step is step 0, so the plots will be generated at steps 4, 9, 14, and 19, which are the 5th, 10th, 15th, and 20th steps.
#     if (step + 1) % 10 == 0:
#         # Plot for the spatial map showing agent locations and adaptation status.
#         plt.figure(figsize=(10, 6))
#         model.plot_model_domain_with_agents()
# 
#         # Plot for the social network showing connections and adaptation statuses.
#         fig, ax = plt.subplots(figsize=(7, 7))
#         plot_network(ax, model)
#         plt.show()

step: 0 : 9 adapted with a bias of 0 and a estimated damage of 1!
step: 3 : 8 adapted with a bias of 0.6000000000000001 and a estimated damage of 0.35547224657343!
step: 4 : 6 adapted with a bias of 0.4 and a estimated damage of 0.53927411832719!
step: 5 : 10 adapted with a bias of 0.4 and a estimated damage of 0.6814665495247537!
step: 6 : 4 adapted with a bias of 0.4 and a estimated damage of 0.6114146391113984!
step: 7 : 14 adapted with a bias of 0.4 and a estimated damage of 0.6594873197404011!
step: 8 : 13 adapted with a bias of 0.4 and a estimated damage of 0.6557668471873622!
step: 8 : 12 adapted with a bias of 0.4 and a estimated damage of 0.5959066918060847!
step: 9 : 39 adapted with a bias of 0.4 and a estimated damage of 0.6921347943585138!
step: 9 : 11 adapted with a bias of 0.6000000000000001 and a estimated damage of 0.5748322319689031!
step: 10 : 20 adapted with a bias of 0.4 and a estimated damage of 0.6823497493915556!
step: 10 : 16 adapted with a bias of 0.2 and a est

In [13]:
agent_data = model.datacollector.get_agent_vars_dataframe()
len([i for i in model.schedule.agents if i.has_child])
agent_data

# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a estimated flood damage of {round(agent.flood_damage_estimated, 3)} and is adapted is {agent.is_adapted} and has conviction: {agent.conviction}\n')
#     
# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a impacted bias because of flooding {round(agent.actual_flood_impact_on_bias, 3)}')

    

FloodDepthEstimated  FloodDamageEstimated  FloodDepthActual  \
Step AgentID                                                                
0    0                   0.183586              0.352341          0.000000   
     1                   0.266617              0.417489          0.000000   
     2                   0.492401              0.524603          0.000000   
     3                   1.792309              0.750180          0.000000   
     4                   0.809566              0.611415          0.000000   
...                           ...                   ...               ...   
19   45                  3.395508              0.861740          2.911130   
     46                  0.000000              0.000000          0.000000   
     47                  1.050894              0.656967          0.635013   
     48                  0.000000              0.000000          0.000000   
     49                  1.727493              0.743749          1.212676   

              FloodDamageActual  IsAdapted  Conviction  FriendsCount  \
Step AgentID                                                           
0    0                 0.000000      False    0.421215             3   
     1                 0.000000      False    0.286380             6   
     2                 0.000000      False    0.410683             2   
     3                 0.000000      False    0.199998             4   
     4                 0.000000      False    0.819154             6   
...                         ...        ...         ...           ...   
19   45                0.834867       True    0.692630             4   
     46                0.000000       True    0.329376             3   
     47                0.569012      False    0.693394             3   
     48                0.000000       True    0.699899             4   
     49                0.681968      False    0.901700             5   

                                                   location  HasChild  \
Step AgentID                                                            
0    0        POINT (223226.36165408476 3315557.2672322504)     False   
     1         POINT (261428.70919878504 3326665.747637199)     False   
     2         POINT (227546.01791107949 3348009.260806306)      True   
     3         POINT (231774.91068052858 3327757.783843969)     False   
     4         POINT (218998.3983496831 3323948.9129254585)     False   
...                                                     ...       ...   
19   45        POINT (219405.80970705827 3328809.826580425)     False   
     46        POINT (225786.34175544517 3327995.504275322)      True   
     47        POINT (252564.68277188475 3292774.660476982)     False   
     48        POINT (247845.9989428391 3285958.0909120278)     False   
     49       POINT (254082.21251367082 3282312.8577439855)     False   

              AgentBias  
Step AgentID             
0    0              0.0  
     1              0.0  
     2              0.0  
     3              0.0  
     4              0.0  
...                 ...  
19   45             2.2  
     46             1.4  
     47             0.0  
     48             1.2  
     49            -0.6  

[1000 rows x 10 columns]

In [5]:
# sum([i.general_bias_in_network for i in model.schedule.agents])/len(model.schedule.agents)

max([i.actual_flood_impact_on_bias for i in model.schedule.agents])

0.027793895941159996

In [30]:
model_data = model.datacollector.get_model_vars_dataframe()
model_data

,total_adapted_households
0,0
1,9
2,9
3,10
4,11
5,11
6,11
7,12
8,12
9,12
